<a href="https://www.kaggle.com/code/search4/black-coffer-data-extraction-nlp?scriptVersionId=142105010" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Name : Archana Yadav
## Objective : to extract textual data articles from the given URL and perform text analysis to compute variables 


In [1]:
pip install syllables

  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.7.0
    Uninstalling importlib-metadata-6.7.0:
      Successfully uninstalled importlib-metadata-6.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-api 1.18.0 requires importlib-metadata~=6.0.0, but you have importlib-metadata 5.2.0 which is incompatible.
yapf 0.40.1 requires importlib-metadata>=6.6.0, but you have importlib-metadata 5.2.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Import required libraries

In [3]:
import numpy as np 
import pandas as pd 
import requests
from bs4 import BeautifulSoup as bs
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import syllables
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize,word_tokenize

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Load data

In [4]:
df = pd.read_excel('/kaggle/input/blackcoffer/Input.xlsx')

In [5]:
df.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


# Extract Title and Text from articles by using Requests and BeautifulSoup

In [6]:
def extraction():
    title_list = []
    main_text = []
    for i in df['URL']:
        try: 
            response =requests.get(i)
            soup =bs(response.text)
            title_list.append(soup.find('h1').text)
            inner_text = []
            for j in soup.find_all('p')[16:-3]:
                inner_text.append(j.text)
            main_text.append(inner_text)
        except:
            title_list.append(np.nan)
            main_text.append(np.nan)
            
    return title_list,main_text
                    

In [7]:
title_list, main_text = extraction()

df['Title'] = title_list
df['Text'] = main_text

In [8]:
df.head()

,URL_ID,URL,Title,Text
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,"[Introduction, “If anything kills over 10 mill..."
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"[Human minds, a fascination in itself carrying..."
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,"[Introduction, AI is rapidly evolving in the e..."
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,[“Anything that could give rise to smarter-tha...
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,[“Machine intelligence is the last invention t...


In [9]:
df.dtypes

URL_ID     int64
URL       object
Title     object
Text      object
dtype: object

In [10]:
df.isna().sum()

URL_ID    0
URL       0
Title     3
Text      3
dtype: int64

### Create copy for NLP operation

In [11]:
nlp_df = df.copy()

# Convert all letters into Lower case

In [12]:
nlp_df['Text']=nlp_df['Text'].apply(lambda x: " ".join(x.lower()  for x in str(x).split()))

In [13]:
nlp_df

,URL_ID,URL,Title,Text
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,"['introduction', '“if anything kills over 10 m..."
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"['human minds, a fascination in itself carryin..."
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,"['introduction', 'ai is rapidly evolving in th..."
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,['“anything that could give rise to smarter-th...
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,['“machine intelligence is the last invention ...
...,...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,['reconciling with the financial realities of ...
110,147,https://insights.blackcoffer.com/the-future-of...,The future of Investing,['an investment is a resource or thing procure...
111,148,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,['quality and affordable healthcare is a visio...
112,149,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,['analytics is a statistical scientific proces...


# Remove Stop Words by using given text files 

In [14]:
import os
files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if os.path.join(dirname, filename).endswith('.txt'):
            files.append(os.path.join(dirname, filename))
files

['/kaggle/input/blackcoffer/StopWords_Currencies.txt',
 '/kaggle/input/blackcoffer/StopWords_GenericLong.txt',
 '/kaggle/input/blackcoffer/StopWords_Currencies-1.txt',
 '/kaggle/input/blackcoffer/StopWords_Names.txt',
 '/kaggle/input/blackcoffer/StopWords_Generic.txt',
 '/kaggle/input/blackcoffer/StopWords_DatesandNumbers.txt',
 '/kaggle/input/blackcoffer/StopWords_Geographic.txt',
 '/kaggle/input/blackcoffer/StopWords_Auditor.txt']

In [15]:
all_stop_words = []
for i in files:
    try : 
        f = open(i,encoding='latin8')
        all_stop_words.append(f.read())
        f.close()
    except:
        print('Not found')

In [16]:
nlp_df['Text'] = nlp_df['Text'].apply(lambda x: " ".join(i for i in x.split() if i not in all_stop_words))   # remove all stopwords
nlp_df['Text']

0      ['introduction', '“if anything kills over 10 m...
1      ['human minds, a fascination in itself carryin...
2      ['introduction', 'ai is rapidly evolving in th...
3      ['“anything that could give rise to smarter-th...
4      ['“machine intelligence is the last invention ...
                             ...                        
109    ['reconciling with the financial realities of ...
110    ['an investment is a resource or thing procure...
111    ['quality and affordable healthcare is a visio...
112    ['analytics is a statistical scientific proces...
113    ['to begin with i shall first like to explain ...
Name: Text, Length: 114, dtype: object

# Perform Lemmatization

In [17]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [18]:
w = WordNetLemmatizer()
nlp_df['Text']=nlp_df['Text'].apply(lambda x:" ".join([w.lemmatize(word) for word in x.split()])) 

# Avg. no. of words per sentence, word count & avg. sentence length

In [19]:
def count_len(len_words,len_sents):
    
    avg_word_len=(len_words)/len_sents
     
    if avg_word_len == 1.0: # for text = nan
        avg_word_length.append(np.nan)
        word_count.append(np.nan)
    else:
        avg_word_length.append(avg_word_len)
        word_count.append(len_words)        # word count
            
    return word_count,avg_word_length,avg_word_len
            

# Complex word count , percentage & fog index

In [20]:
def complex_words_fogidx(word_list,len_words,avg_word_len):
    complex_word_count=0
    for w in word_list:
        vc=0 #vowelcount
        for v in vowels:
            if v in w:
                vc+=1
        if vc>1:
            complex_word_count+=1

    if complex_word_count == 0:
        percent_complex_word.append(np.nan)
        complex_word.append(np.nan)              
 
    else:
        p_cw = (complex_word_count/len_words)*100
        
        complex_word.append(complex_word_count)         # complex word count
        
        percent_complex_word.append(p_cw)    
            
            #----------fog index--------
            
    if avg_word_len == 1.0 and complex_word_count == 0:
            fog_index.append(np.nan)
    else:
        fog_idx=0.4*(avg_word_len+10*(p_cw))    ##0.4 [(words/sentences) + 10 (complex words/words)]
        fog_index.append(round(fog_idx,2))
        
    return percent_complex_word,complex_word,fog_index      

# Syllables count

In [21]:
def syllables_word(word_list):
    syl_count = 0
    for word in word_list:
        syl_count+=syllables.estimate(word)
    if syl_count==1:
        syl_count_sum.append(np.nan)
    else:
        syl_count_sum.append(syl_count)
        
    return syl_count_sum

# Personal Pronoun

In [22]:
def personal_pronouns(word_list):
    count=0
    for i,j in (pos_tag(word_list)):
        if j=='PRP':
            count+=1
    personal_pronoun.append(np.sum(count))
    
    return personal_pronoun

# Avg. word length

In [23]:
def avgwordlen(word_list,len_words):
    count1=0
    for w in word_list:
        cpw=len(w)
        count1+=cpw
    avg_word_len=count1/len_words
    
    if avg_word_len == 3:
        avg_word.append(np.nan)
    else:
        avg_word.append(avg_word_len)
        
    return avg_word

# Positive score, Negative score, Polarity, Subjectivity

In [24]:
def pos_neg_scores(word_list):
    neg,pos,pol=0,0,0
    for i in word_list:
        sid_obj = SentimentIntensityAnalyzer()
    
        neg += sid_obj.polarity_scores(i)['neg']
        pos += sid_obj.polarity_scores(i)['pos']
        pol += sid_obj.polarity_scores(i)['compound']
        
    subjectivity=(pos+neg)/len_words
        
        
    neg_score.append(neg)
    pos_score.append(pos)
    pol_score.append(pol)
    sub_score.append(subjectivity)
        
    return neg_score,pos_score,pol_score,sub_score


In [25]:
avg_word_length = []
percent_complex_word = []
fog_index = []
sent_count = []
complex_word = []
word_count= []
syl_count_sum = []
personal_pronoun = []
avg_word = []
vowels='aeiou'
pos_score = []
neg_score =[]
pol_score = []
sub_score = []

In [26]:
for i in range(len(nlp_df)):
    
        text=nlp_df['Text'][i]
        without_punctuation = ''.join(re.findall(r'[a-zA-Z+" "]',text))     # removing punctuation
        
        word_list = word_tokenize(without_punctuation)   ## Word tokenize applied after removing punctuation
        len_words=len(word_list) 
                
        len_sents=len(sent_tokenize(text))           # sentence tokenize applied on text before removing punctuation
        
        
        #----------Avg word per sent., word count, sent. Length--------------------------


        word_count,avg_word_length,avg_word_len=count_len(len_words,len_sents)  
        
            
        #----------------complex word,  fog index-----------------    
            
        complex_words_fogidx(word_list,len_words,avg_word_len)  
        
       
            
        #---------------Syllables-------------------

        syllables_word(word_list)
        
            
        #-------------PERSONAL PRONOUNS----------------
        
        personal_pronouns(word_list)
        
        
        #-----------AVERAGE WORD LENGTH---------------
        
        avgwordlen(word_list,len_words)
        
        
        #-----------Positive score, negative score, polarity, subjectivity-------------

        pos_neg_scores(word_list)
        

In [27]:
output =pd.read_excel('/kaggle/input/blackcoffer/Output Data Structure.xlsx')

In [28]:
output['POSITIVE SCORE'] = pos_score
output['NEGATIVE SCORE'] = neg_score
output['POLARITY SCORE']=  pol_score
output['SUBJECTIVITY SCORE'] = sub_score
output['AVG SENTENCE LENGTH'] = avg_word_length
output['PERCENTAGE OF COMPLEX WORDS'] = percent_complex_word
output['FOG INDEX'] = fog_index
output['AVG NUMBER OF WORDS PER SENTENCE'] = avg_word_length
output['COMPLEX WORD COUNT'] = complex_word
output['WORD COUNT'] = word_count
output['SYLLABLE PER WORD'] = syl_count_sum
output['PERSONAL PRONOUNS'] = personal_pronoun
output['AVG WORD LENGTH']= avg_word

In [29]:
output

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,92.0,28.0,22.3024,0.067912,24.205479,55.291454,230.85,24.205479,977.0,1767.0,3504.0,25,5.571590
1,38,https://insights.blackcoffer.com/what-if-the-c...,84.0,40.0,13.7633,0.088889,18.600000,44.157706,184.07,18.600000,616.0,1395.0,2356.0,39,4.752688
2,39,https://insights.blackcoffer.com/what-jobs-wil...,74.0,36.0,11.2381,0.065398,20.512195,51.605232,214.63,20.512195,868.0,1682.0,3263.0,29,5.310939
3,40,https://insights.blackcoffer.com/will-machine-...,100.0,28.0,26.5754,0.081373,17.097826,47.171011,195.52,17.097826,742.0,1573.0,2704.0,54,4.735537
4,41,https://insights.blackcoffer.com/will-ai-repla...,98.0,28.0,25.6136,0.075767,25.984375,47.143716,198.97,25.984375,784.0,1663.0,2995.0,49,5.005412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,43.0,20.0,7.8205,0.072581,18.083333,50.921659,210.92,18.083333,442.0,868.0,1635.0,19,5.396313
110,147,https://insights.blackcoffer.com/the-future-of...,49.0,11.0,14.7555,0.056075,22.291667,50.000000,208.92,22.291667,535.0,1070.0,1979.0,15,5.137383
111,148,https://insights.blackcoffer.com/big-data-anal...,49.0,40.0,0.1388,0.080108,18.213115,46.354635,192.70,18.213115,515.0,1111.0,2050.0,14,5.024302
112,149,https://insights.blackcoffer.com/business-anal...,45.0,3.0,15.6551,0.089219,25.619048,51.115242,214.71,25.619048,275.0,538.0,1093.0,8,5.745353
